## 연합뉴스 타이틀 주제 분류
* 데이터셋 출처 : 
    * [뉴스 토픽 분류 AI 경진대회 - DACON](https://dacon.io/competitions/official/235747/overview/description)

## 기초 분류 모델 만들기

* 데이터 로드
* 데이터 전처리
* 단어 벡터화(BOW, TF-IDF)
* 분류기 설정하기
* 분류기로 학습시키기
* 학습의 정확도 보기
* 테스트 데이터 예측하기
* 실제 데이터와 예측결과의 차이를 보기

<img src="https://i.imgur.com/Sxm12BY.png" width="900">



## 라이브러리 로드

In [ ]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 시각화를 위한 폰트 설정

In [ ]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family


# style 설정은 꼭 폰트설정 위에서 합니다.
# style 에 폰트 설정이 들어있으면 한글폰트가 초기화 되어 한글이 깨집니다.
plt.style.use("seaborn")
# 폰트설정
plt.rc("font", family=get_font_family())

# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

In [ ]:
pd.Series([1, 3, 5, -7, 9]).plot(title="한글", figsize=(5, 2))

## 데이터 로드

In [ ]:
# 데이콘의 해당 데이터셋은 CC-BY-4.0 라이센스입니다.
# 데이터 출처 : https://dacon.io/competitions/official/235747/data
# 로컬 PC에서 실습 시 직접 데이콘 사이트에 회원가입하고 다운로드 해주세요.

import os, platform
base_path = "data/klue/"

def file_exist_check(base_path):
    if os.path.exists(f"{base_path}train_data.csv"):
        print(f"{base_path} 경로에 파일이 이미 있음")
        return
    
    if platform.system() == "Linux":
        print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
        !wget https://bit.ly/dacon-klue-open-zip
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        !unzip dacon-klue-open-zip -d data/klue
    else:
        print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 하고
              실습 경로 {base_path}에 옮겨주세요.""")
    return
    
file_exist_check(base_path) 

In [ ]:
# 학습, 예측 데이터셋을 불러옵니다.
train = pd.read_csv("data/klue/train_data.csv")
test = pd.read_csv("data/klue/test_data.csv")
train.shape, test.shape

In [ ]:
# 토픽을 불러옵니다.
topic = pd.read_csv("data/klue/topic_dict.csv")
topic

## 문자 전처리

In [ ]:
# 정규표현식
import re

def preprocessing(text):
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub('[\s]+', ' ', text)
    # 영문자를 소문자로 만듭니다.
    text = text.lower()
    return text

In [ ]:
# map을 통해 전처리 일괄 적용
train["title"] = train["title"].map(preprocessing)
test["title"] = test["title"].map(preprocessing)

## 형태소 분석
* [KoNLPy: 파이썬 한국어 NLP — KoNLPy documentation](https://konlpy.org/ko/latest/)
    * Hannanum Class
    * Kkma Class
    * Komoran Class
    * Mecab Class
    * Okt Class

```sh
pip install --upgrade pip
pip install JPype1
pip install konlpy --upgrade
```

다음의 항목을 만족해야 konlpy를 윈도우나 맥에서 사용할 수 있습니다.
대부분의 오류는 환경변수나 최신버전의 JDK가 설치되지 않아서 입니다. 공식문서에서는 1.7 이상의 JDK 를 권장하고 있습니다.
은전한닢(mecab)은 별도의 설치가 필요합니다.

```
1) 최신 버전의 JAVA(JDK)를 설치
2) JAVA_HOME 환경변수를 추가
3) path 환경변수에 %JAVA_HOME%\bin; 추가
```

설치 관련 자세한 내용은 다음을 참고해 보세요.

https://konlpy.org/ko/latest/install/

In [ ]:
# !pip install konlpy --upgrade

In [ ]:
small_text = "버스의 운행시간을 문의합니다. 어?!"
small_text

In [ ]:
# Okt
from konlpy.tag import Okt

Okt().pos(small_text, stem=True)

In [ ]:
# !pip install tqdm --upgrade

In [ ]:
# 형태소 분석기(Okt) 불러오기 

from konlpy.tag import Okt
okt = Okt() 

# 조사, 어미, 구두점 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True): # 어간 추출
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']: # 조사, 어미, 구두점 제외 
            clean_text.append(word[0])
    
    return " ".join(clean_text) 

In [ ]:
# !pip install tqdm --upgrade

In [ ]:
from tqdm import tqdm
tqdm.pandas() 

train['title'] = train['title'].progress_map(okt_clean)
test['title'] = test['title'].progress_map(okt_clean)

## 불용어 제거

In [ ]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['그', '등', '이런', '및','제', '더']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [ ]:
train["title"] = train["title"].map(remove_stopwords)
test["title"] = test["title"].map(remove_stopwords)
train["title"]

## 학습, 예측 데이터셋 만들기

In [ ]:
X_train_text = train["title"]
X_test_text = test["title"]

In [ ]:
# 정답값으로 사용할 topic_idx 를 변수에 담아 재사용 합니다.
label_name = "topic_idx"

In [ ]:
# 기출문제의 정답을 만들어 줍니다.
# 잘 나뉘었는지 정닶 값의 빈도수를 확인해 봅니다.
y_train = train[label_name]
y_train.value_counts()

## 벡터화
* 머신러닝이나 딥러닝 알고리즘은 문자를 이해할 수 없습니다. 내부에서는 수치 계산이 이루어지기 때문에 문자를 숫자로 변경해 주어야 합니다.


### TF-IDF(Term Frequency - Inverse Document Frequency)

In [ ]:
# TfidfVectorizer 로 벡터화 합니다.
# fit 으로 변환할 어휘를 학습합니다.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvect = TfidfVectorizer(tokenizer=None, 
                             ngram_range=(1,2),
                             min_df=2, 
                             max_df=0.95)
tfidfvect.fit(X_train_text)

In [ ]:
# transform
X_train = tfidfvect.transform(X_train_text)
X_test = tfidfvect.transform(X_test_text)

X_train.shape, X_test.shape

## 모델


전체 과정
* feature_names : 학습(훈련), 예측에 사용할 단어 사전 입니다.
* label_name : 정답값

* X_train : feature_names 에 해당되는 컬럼만 train에서 가져옵니다.
    * 학습(훈련)에 사용할 데이터셋 예) 시험의 기출문제

* X_test : feature_names 에 해당되는 컬럼만 test에서 가져옵니다.
    * 예측에 사용할 데이터셋 예) 실전 시험문제

* y_train : label_name 에 해당 되는 컬럼만 train에서 가져옵니다.
    * 학습(훈련)에 사용할 정답 값 예) 기출문제의 정답

* model : 학습, 예측에 사용할 머신러닝 알고리즘

* model.fit(X_train, y_train) : 학습(훈련), 기출문제와 정답을 가지고 학습(훈련)하는 과정과 유사합니다.

* model.predict(X_test) :  예측, 실제 시험을 보는 과정과 유사합니다. => 문제를 풀어서 정답을 구합니다.

* score
    * 시험을 봤다면 몇 문제를 맞고 틀렸는지 채점해 봅니다.
* metric
    * 점수를 채점하는 공식입니다. (예를 들어 학교에서 중간고사를 봤다면 전체 평균을 계산해 줍니다.)
    
## 학습과 예측


### DecisionTree
* 결정트리 모델
* https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

### RandomForest
* 앙상블 모델 중 배깅을 이용한 모델
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

### GradientBoosting
* 앙상블 모델 중 부스팅을 이용한 모델
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

### ExtraTrees
* 앙상블 모델 중 엑스트라 트리를 이용한 모델
* 랜덤 포레스트에서와 같이 후보 기능의 무작위 하위 집합이 사용되지만 가장 차별적인 임계값을 찾는 대신 각 후보 기능에 대해 임계값이 무작위로 그려지고 무작위로 생성된 임계값 중 가장 좋은 것이 분할 규칙으로 선택됩니다. 이것은 일반적으로 약간 더 큰 편향 증가를 희생시키면서 모델의 분산을 조금 더 줄일 수 있습니다.
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

### XGBoost
* pip 설치 : [Installation Guide — xgboost documentation](https://xgboost.readthedocs.io/en/stable/install.html)
* 아나콘다 환경 설치 : `conda install -c conda-forge xgboost`
* Gradient Boosting Tree에서 병렬 학습을 지원하는 모델을 구현한 라이브러리입니다.
* 특징
    - GBM 대비 빠른 수행시간(병렬 처리)
    - 과적합 규제(Regularization)
    표준 GBM 경우 과적합 규제기능이 없으나, XGBoost는 자체에 과적합 규제 기능으로 강한 내구성을 지님
    - 분류와 회귀에서 뛰어난 예측 성능 
    - Early Stopping(조기 종료) 기능이 있음
    - 다양한 옵션(Hyper Parameter)을 제공하며 Customizing이 용이


*  https://xgboost.readthedocs.io/en/stable/

### LightGBM
* 설치 : [Installation Guide — LightGBM documentation](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html)
* XGBoost와 성능은 비슷하지만 시간과 메모리 측면에서 더 효율적인 모델을 구현한 라이브러리입니다.
* Light Gradient Boosting Machine의 약자인 LightGBM 은 Microsoft 에서 개발한 머신 러닝을 위한 무료 오픈 소스 분산 그라디언트 부스팅 프레임워크입니다. 결정 트리 알고리즘을 기반으로 하며 순위 지정, 분류 및 기타 기계 학습 작업에 사용합니다.
* LightGBM은 효율성과 메모리 소비 모두에서 큰 이점을 제공하는 고도로 최적화된 히스토그램 기반 의사결정 트리 학습 알고리즘을 구현합니다. LightGBM 알고리즘은 GOSS(Gradient-Based One-Side Sampling) 및 EFB(Exclusive Feature Bundling)라는 두 가지 기술을 사용하여 높은 수준의 정확도를 유지하면서 알고리즘을 더 빠르게 실행할 수 있습니다.
* 기울기 기반 단측 샘플링 : GOSS(Gradient-Based One-Side Sampling)는 GBDT에서 데이터 인스턴스에 대한 기본 가중치가 없다는 것을 활용하는 방법입니다. 기울기가 다른 데이터 인스턴스는 정보 이득 계산에서 다른 역할을 하므로 기울기가 더 큰 인스턴스는 정보 이득에 더 많이 기여할 것입니다. 따라서 정보의 정확성을 유지하기 위해 GOSS는 기울기가 큰 인스턴스를 유지하고 기울기가 작은 인스턴스를 무작위로 삭제합니다.
* 배타적 특성 묶음 : EFB(Exclusive Feature Bundling)는 유효 기능의 수를 줄이는 거의 무손실 방법입니다. 희소 특성 공간에서 많은 특성은 거의 배타적이어서 0이 아닌 값을 동시에 취하는 경우가 거의 없음을 의미합니다. 원-핫 인코딩 기능이 배타적 기능의 예입니다. EFB는 이러한 기능을 번들로 제공하여 높은 수준의 정확도를 유지하면서 효율성을 개선하기 위해 차원을 줄입니다. 배타적 기능을 단일 기능으로 묶는 것을 배타적 기능 번들이라고 합니다.


* [LightGBM 고효율 그래디언트 부스팅 결정 트리 논문 번역 블로그](https://aldente0630.github.io/data-science/2018/06/29/highly-efficient-gbdt.html)


* https://lightgbm.readthedocs.io/en/latest/Features.html
* https://lightgbm.readthedocs.io

### catboost
* Gradient Boosting Tree에서 과적합 방지와 범주형 변수에 대해 강점을 가진 모델을 구현한 라이브러리입니다.
*  https://catboost.ai/

In [ ]:
# 아나콘다 사용시 conda 설치 권장, pip 는 colab 에서 설치 시 사용하세요.
# conda install -c conda-forge xgboost
# pip install xgboost

# xgbc

In [ ]:
# 아나콘다 사용시 conda 설치 권장, pip 는 colab 에서 설치 시 사용하세요.
# conda install -c conda-forge lightgbm
# pip install lightgbm

# lgbmc

## 교차 검증

<img src="https://i.imgur.com/x9M8zpI.png" width="600">

In [ ]:
# cross_val_predict 로 교차 검증을 합니다.


### 교차 검증 정확도

In [ ]:
# 교차 검증 결과 


In [ ]:
# 교차 검증 결과 점수를 구해봅니다.
# 모의고사를 보고 채점해 보는 과정과 유사합니다.


In [ ]:
# df_accuracy 로 정답여부 컬럼을 만듦


In [ ]:
# groupby 로 분류별 정확도를 구합니다.


In [ ]:
# pd.crosstab


In [ ]:
# confusion_matrix


## 모델 설명하기

In [ ]:
# xgboost feature_importances_


In [ ]:
# xgboost plot_importance


In [ ]:
# xgboost plot_tree


In [ ]:
# xgboost to_graphviz


In [ ]:
# lightgbm create_tree_digraph


## 예측

In [ ]:
# 실전 시험과 유사합니다. 정답 없이 문제만 넣고 풀어서 예측합니다.
# predict로 예측합니다. 


In [ ]:
# 실제값과 예측값의 빈도 비율


## 정답값 로드

In [ ]:
# sample_submission 파일을 불러옵니다.
# Dacon 에 제출할 답안지 양식입니다.
submit = pd.read_csv("data/klue/sample_submission.csv")
submit.head()

In [ ]:
# 정답값 측정을 위해 y_test 변수에 할당
# submit["topic_idx"]

In [ ]:
# valid_accuracy

In [ ]:
file_name = f"data/klue/submit_{valid_accuracy}.csv"
file_name

In [ ]:
# 잘 저장이 되었는지 확인합니다.


In [ ]:
# 잘 저장이 되었는지 확인합니다.


* dacon에 제출해서 리더보드 확인하기 : https://dacon.io/competitions/official/235747/mysubmission


## 여러 방법을 통해 예측비율을 높여보세요. 
* 아래 항목 외의 기법을 사용해도 됩니다.
 * 전처리 하기
 * 불용어 처리
 * BOW, TF-IDF의 파라메터 변경
 * 분류기의 파라메터 변경
 * 분류기 변경